Source: https://github.com/dsgermain/gbfs/blob/f76251ad4c754b62defc42562887724f287b73ea/gbfs.md#file-requirements

In [376]:
# import libraries
import requests
import pandas as pd
import datetime 
from google.cloud import bigquery
import pytz

ImportError: cannot import name 'bigquery' from 'google.cloud' (unknown location)

In [369]:
# empty strings get list of urls
name_list = []
url_list = []

url = "https://mds.linkyour.city/gbfs/us_tx_austin/gbfs.json"
response = requests.get(url).json()
for i in response['data']['en']['feeds']:
    name_list.append(i['name'])
    url_list.append(i['url'])

url_df = pd.DataFrame({'name' : name_list, 'url' : url_list})

try: 
    fbs_url = url_df.loc[url_df['name'] == 'free_bike_status']['url'].values[0]
except IndexError:
    print(f"ERROR: free_bike_status file does not exist for the url: {url}")
else:
    fbs_url = fbs_url.replace("'", "")
    fbs_response = requests.get(fbs_url).json()

In [370]:
fbs_response['last_updated']
# , fbs_response['ttl']

1653022551

In [372]:
bike_dict = fbs_response['data']['bikes']
bike_df = pd.DataFrame.from_dict(bike_dict)

last_updated = datetime.datetime.fromtimestamp(int(fbs_response['last_updated'])).strftime('%Y-%m-%d %H:%M:%S')
bike_df['last_updated'] = last_updated
bike_df.head()

,bike_id,vehicle_type_id,lat,lon,is_reserved,is_disabled,pricing_plan_id,last_updated_ct
0,c8dc417a-2d21-471f-ab26-20130a31159a,1,30.26005,-97.75452,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-19 23:55:51
1,f1e2d98c-0384-4151-b2b4-eff24d4ad362,1,30.29067,-97.74290,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-19 23:55:51
2,86d16290-e3fc-44a4-9974-50bdf5f7f279,1,30.24773,-97.73360,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-19 23:55:51
3,11fa32b8-f392-430f-863f-01ae94ec94fa,1,30.25688,-97.74669,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-19 23:55:51
4,391726be-4841-4482-a698-c82a943815c2,1,30.26340,-97.74457,False,False,9ff58a49-0549-48e3-b57c-d7cf2c453704,2022-05-19 23:55:51


In [373]:
# Reference: https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe

# Construct a BigQuery client object.
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job = client.load_table_from_dataframe(bike_df, 'jintaepark-portoflio-project.bikeshare_analytics.test_upload', job_config=job_config)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

NameError: name 'bigquery' is not defined